In [7]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential
from azure.ai.ml import MLClient
import configparser
import os

# Load environment variables from config.ini file
config = configparser.ConfigParser()

# Use an absolute path to avoid path issues
config_file_path = "G:/My Drive/Ingegneria/Data Science GD/My-Practice/my_models/Azure ML/config.ini"
config.read(config_file_path)

# Check if the 'azure' section exists
if 'azure' in config:
    os.environ['AZURE_CLIENT_ID'] = config['azure']['client_id']
    os.environ['AZURE_CLIENT_SECRET'] = config['azure']['client_secret']
    os.environ['AZURE_TENANT_ID'] = config['azure']['tenant_id']
    os.environ['AZURE_SUBSCRIPTION_ID'] = config['azure']['subscription_id']

    # Attempt to use DefaultAzureCredential
    try:
        credential = DefaultAzureCredential()
        credential.get_token("https://management.azure.com/.default")
    except Exception as ex:
        print("DefaultAzureCredential failed, falling back to InteractiveBrowserCredential.")
        credential = InteractiveBrowserCredential()

    # Initialize MLClient with the obtained credential
    ml_client = MLClient(
        credential=credential,
        subscription_id=os.environ['AZURE_SUBSCRIPTION_ID'],
        resource_group_name="rg-dp100-labs",
        workspace_name="mlw-dp100-labs"
    )

    # List workspaces to verify the connection
    try:
        workspaces = ml_client.workspaces.list()
        for ws in workspaces:
            print(ws.name)
    except Exception as e:
        print(f"Failed to list workspaces: {e}")

else:
    print("The 'azure' section is missing in the config.ini file.")


After defining the authentication, you need to call MLClient for the environment to connect to the workspace. You'll call MLClient anytime you want to create or update an asset or resource in the workspace.

In [11]:
!git clone https://github.com/MicrosoftLearning/mslearn-azure-ml.git azure-ml-labs

Cloning into 'azure-ml-labs'...
Updating files:  75% (75/99)
Updating files:  76% (76/99)
Updating files:  77% (77/99)
Updating files:  78% (78/99)
Updating files:  79% (79/99)
Updating files:  80% (80/99)
Updating files:  81% (81/99)
Updating files:  82% (82/99)
Updating files:  83% (83/99)
Updating files:  84% (84/99)
Updating files:  85% (85/99)
Updating files:  86% (86/99)
Updating files:  87% (87/99)
Updating files:  88% (88/99)
Updating files:  89% (89/99)
Updating files:  90% (90/99)
Updating files:  91% (91/99)
Updating files:  92% (92/99)
Updating files:  93% (93/99)
Updating files:  94% (94/99)
Updating files:  95% (95/99)
Updating files:  96% (96/99)
Updating files:  97% (97/99)
Updating files:  98% (98/99)
Updating files: 100% (99/99)
Updating files: 100% (99/99), done.


In [9]:
# example better detailed later on

'''

# For example, you'll connect to the workspace when you create a new job to train a model:

# configure job
job = command(
    code="./src",
    command="python train.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    experiment_name="train-model"
)

# connect to workspace and submit job
returned_job = ml_client.create_or_update(job)

'''